<a href="https://colab.research.google.com/github/Joab-S/SimulacaoEstocasticaUFC/blob/main/T5_SimEstoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import random as r
import simpy as sp
import numpy as np
import matplotlib.pyplot as mp
import itertools

Q1) Construção de Produtos

Q2) Restaurante

Q3) Supermercado

In [71]:
# SIMULAÇÃO EM HORAS
BANCO_DE_CLIENTES = {}
TOTAL_CLIENTES_NO_MERCADO = 0

class Client():
  newid = itertools.count().__next__
  HORA_CHEGADA: float;
  HORA_SAIDA: float;
  DIA_DA_COMPRA: float;
  TEMPO_DE_COMPRA: float;
  TEMPO_DE_ATENDIMENTO: float;
  FOI_ATENDIDO: bool = False

  def __init__(self, hC, dC):
    self.id = Client.newid()
    self.HORA_CHEGADA = hC;
    self.DIA_DA_COMPRA = dC;

  def getID(self):
    return self.id;

  def setTempoDeCompra(self, t):
    self.TEMPO_DE_COMPRA = t;

  def getTempoDeCompra(self):
    return self.TEMPO_DE_COMPRA
  
  def setHoraSaida(self, t):
    self.HORA_SAIDA = t;

  def getHoraSaida(self):
    return self.HORA_SAIDA

  def setAtendido(self):
    self.FOI_ATENDIDO = -1*self.FOI_ATENDIDO

  def foiAtendido(self):
    return self.FOI_ATENDIDO

class Turno():
  TURNO: str;
  TAXA_CHEGADA: float;
  CAIXA: int;
  MIN_TEMPO_COMPRA: float;
  MAX_TEMPO_COMPRA: float;
  N_CLIENTES_C = 0
  N_CLIENTES_S = 0

  def __init__(self, turno, taxa_c, caixa, min_compra, max_compra):
    self.TURNO = turno;
    self.TAXA_CHEGADA = taxa_c;
    self.CAIXA = caixa;
    self.MIN_TEMPO_COMPRA = min_compra;
    self.MAX_TEMPO_COMPRA = max_compra;

  def setN_clientes_C(self, n):
    self.N_CLIENTES_C = n;

  def setN_clientes_S(self, n):
    self.N_CLIENTES_S = n;

  def getN_clientes_C(self):
    return self.N_CLIENTES_C;

  def getN_clientes_S(self):
    return self.N_CLIENTES_S;

TURNO_1  = Turno ("08h - 16h", 16.8, 2,   5, 15)
TURNO_2  = Turno ("16h - 00h", 24.0, 4,  15, 40)
TURNO_3  = Turno ("00h - 08h",  0.7, 1,   1,  5)
LIMITE = 720                                          # 30 dias em horas
UNIDADE_TEMPO = 24

def initVar():
  global TURNO_1, TURNO_2, TURNO_3, LIMITE

  TURNO_1  = Turno ("08h - 16h", 16.8, 2,   5, 15)
  TURNO_2  = Turno ("16h - 00h", 24.0, 4,  15, 40)
  TURNO_3  = Turno ("00h - 08h",  0.7, 1,   1,  5)
  LIMITE = 720                                        # 760 - horas   |   43200 - minutos
  UNIDADE_TEMPO = 24                                  # 24  - horas   |   1440  - minutos


def getDia(env):
  """Retorna o dia da simulação"""

  global LIMITE, UNIDADE_TEMPO
  total_dia = LIMITE/UNIDADE_TEMPO

  dias_restantes = (LIMITE - env.now)/UNIDADE_TEMPO
  dia = total_dia - dias_restantes
  
  return dia


def getTurno(env) -> Turno:
  global TURNO_1, TURNO_2, TURNO_3, UNIDADE_TEMPO
  
  DIA = getDia(env)
  _8h = UNIDADE_TEMPO/3
  
  if (env.now <= (_8h + 24*int(DIA))):
    return TURNO_3

  if (env.now > (_8h + 24*int(DIA)) and env.now <= (_8h*2 + 24*int(DIA))):
    return TURNO_1

  if (env.now > (_8h*2 + 24*int(DIA)) and env.now <= (_8h*3 + 24*int(DIA))):
    return TURNO_2


def chegada(env, turno: Turno):
  global TOTAL_CLIENTES_NO_MERCADO

  while True:
    yield env.timeout(np.random.poisson(10/(turno.TAXA_CHEGADA)))

    turno = getTurno(env)
    turno.setN_clientes_C(turno.getN_clientes_C() + 1)
    dia = int(getDia(env))
    hora_atual = env.now - UNIDADE_TEMPO*dia
    client = Client(hora_atual, dia+1)
    BANCO_DE_CLIENTES[client.getID()] = client
    
    TOTAL_CLIENTES_NO_MERCADO += 1

    print(f'Um novo cliente de id "{client.getID()}" chegou no turno de {turno.TURNO}, às {hora_atual}h do {dia+1}º dia do mês.')
    print(f'Um total de {turno.getN_clientes_C()} clientes chegaram nesse turno. Tempo de simulação em {env.now}.')
    print(f'Há um total de {TOTAL_CLIENTES_NO_MERCADO} clientes no Supermercado.')
    print()

    env.process(compra(env, client.getID(), turno))
  

def compra(env, clientID, turno: Turno):
  global TOTAL_CLIENTES_NO_MERCADO

  tCompra = np.random.uniform(turno.MIN_TEMPO_COMPRA/60, turno.MAX_TEMPO_COMPRA/60)
  yield env.timeout(tCompra)

  cliente = BANCO_DE_CLIENTES[clientID]
  cliente.setTempoDeCompra(tCompra)

  print(f'O cliente "{cliente.getID()}" passou %.0f minutos comprando entre {turno.TURNO}.' %(tCompra*60))


  #if (TOTAL_CLIENTES_NO_MERCADO == 1):
  #  env.process(atendimento(env, turno))


def atendimento(env, turno: Turno):
  global TOTAL_CLIENTES_NO_MERCADO

  while (TOTAL_CLIENTES_NO_MERCADO > 0):
    cliente = min(BANCO_DE_CLIENTES, key=lambda d : d['TEMPO_DE_COMPRA'])
    
    tAtendimento = np.random.triangular(0.2*cliente.TEMPO_DE_COMPRA, )
    yield env.timeout(tAtendimento)
    
    TOTAL_CLIENTES_NO_MERCADO -= 1


"""
def initSystem(env):
  initVar()

  env.process(arrivalSystem(env))
  if (client_number > 0):
    env.process(occurrence(env))
    env.process(exitSystem(env))

"""


def systemSim():
  global LIMITE

  initVar()
  env = sp.Environment()
  turno = getTurno(env)
  env.process(chegada(env, turno))
  env.run(until=LIMITE)

systemSim()

Um novo cliente de id "0" chegou no turno de 00h - 08h, às 5h do 1º dia do mês.
Um total de 1 clientes chegaram nesse turno. Tempo de simulação em 5.
Há um total de 1 clientes no Supermercado.

O cliente "0" passou 3 minutos comprando entre 00h - 08h.
Um novo cliente de id "1" chegou no turno de 00h - 08h, às 0h do 2º dia do mês.
Um total de 2 clientes chegaram nesse turno. Tempo de simulação em 24.
Há um total de 2 clientes no Supermercado.

O cliente "1" passou 4 minutos comprando entre 00h - 08h.
Um novo cliente de id "2" chegou no turno de 08h - 16h, às 12h do 2º dia do mês.
Um total de 1 clientes chegaram nesse turno. Tempo de simulação em 36.
Há um total de 3 clientes no Supermercado.

Um novo cliente de id "3" chegou no turno de 08h - 16h, às 12h do 2º dia do mês.
Um total de 2 clientes chegaram nesse turno. Tempo de simulação em 36.
Há um total de 4 clientes no Supermercado.

O cliente "2" passou 10 minutos comprando entre 08h - 16h.
O cliente "3" passou 13 minutos comprando en